# 2. Preprocessing & Feature Engineering

## Load Libraries and Data

In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully.")

Libraries loaded successfully.


In [45]:
# Load raw data
df = pd.read_csv('../data/shipment_booking_data_2021_2025.csv')
df['booking_date'] = pd.to_datetime(df['booking_date'])

print(f"Loaded {len(df):,} records")
print(f"Date range: {df['booking_date'].min().date()} to {df['booking_date'].max().date()}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
display(df.head())

Loaded 206,559 records
Date range: 2021-01-01 to 2025-12-31

Columns: ['company_name', 'shipment_type', 'booking_date']

First few rows:


,company_name,shipment_type,booking_date
0,BlueDart,Express,2021-01-01
1,BlueDart,Surface,2021-01-01
2,BlueDart,Surface,2021-01-01
3,BlueDart,Air,2021-01-01
4,BlueDart,Surface,2021-01-01


## Step 1: Daily Aggregation

Aggregate bookings by:
- Company name
- Booking date (daily frequency)
- Shipment type counts

In [46]:
# Daily aggregation: Count each shipment type per company per day
df_daily = df.set_index('booking_date').groupby([
    'company_name', 
    pd.Grouper(freq='D')
])['shipment_type'].value_counts().unstack(fill_value=0).reset_index()

# Clean column names
df_daily.columns.name = None

print(f"Daily aggregated data shape: {df_daily.shape}")
print(f"\nColumns after aggregation: {list(df_daily.columns)}")
print(f"\nFirst few rows:")
display(df_daily.head(10))

Daily aggregated data shape: (14607, 6)

Columns after aggregation: ['company_name', 'booking_date', 'Air', 'Express', 'International', 'Surface']

First few rows:


,company_name,booking_date,Air,Express,International,Surface
0,BlueDart,2021-01-01,5,2,1,6
1,BlueDart,2021-01-02,4,1,2,5
2,BlueDart,2021-01-03,6,2,0,5
3,BlueDart,2021-01-04,3,3,1,7
4,BlueDart,2021-01-05,7,0,1,3
5,BlueDart,2021-01-06,2,3,0,8
6,BlueDart,2021-01-07,3,1,0,4
7,BlueDart,2021-01-08,4,2,0,7
8,BlueDart,2021-01-09,9,1,0,6
9,BlueDart,2021-01-10,7,0,2,3


In [47]:
# Define target columns (shipment types)
target_columns = ['Air', 'Express', 'International', 'Surface']

# Ensure all target columns exist (fill with 0 if missing)
for col in target_columns:
    if col not in df_daily.columns:
        df_daily[col] = 0
        print(f"Added missing column: {col}")

print(f"\nTarget columns verified: {target_columns}")


Target columns verified: ['Air', 'Express', 'International', 'Surface']


In [48]:
# Sort by company and date (CRITICAL for lag features)
df_daily = df_daily.sort_values(['company_name', 'booking_date']).reset_index(drop=True)

print("Data sorted by company_name and booking_date.")
print(f"Total rows: {len(df_daily):,}")

Data sorted by company_name and booking_date.
Total rows: 14,607


## Step 2: Feature Engineering

Create features for each shipment type:
- **lag_1**: Today's actual volume (shift 0)
- **lag_7**: Volume from 7 days ago (shift 6)
- **roll_7**: 7-day rolling average

In [49]:
# Initialize feature list
feature_cols = []

print("Creating lag and rolling features for each shipment type...\n")

for col in target_columns:
    print(f"Processing: {col}")
    
    # LAG FEATURES
    # lag_1: Today's value (shift 0 means current day)
    df_daily[f'lag_1_{col}'] = df_daily.groupby('company_name')[col].shift(0)
    feature_cols.append(f'lag_1_{col}')
    
    # lag_7: 7 days ago (shift 6 because we want 7th day back including today)
    df_daily[f'lag_7_{col}'] = df_daily.groupby('company_name')[col].shift(6)
    feature_cols.append(f'lag_7_{col}')
    
    # ROLLING FEATURES
    # roll_7: 7-day rolling mean (includes current day)
    df_daily[f'roll_7_{col}'] = df_daily.groupby('company_name')[col].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )
    feature_cols.append(f'roll_7_{col}')

print(f"\nCreated {len(feature_cols)} lag/rolling features")
print(f"Feature list: {feature_cols[:6]}... (showing first 6)")

Creating lag and rolling features for each shipment type...

Processing: Air
Processing: Express
Processing: International
Processing: Surface

Created 12 lag/rolling features
Feature list: ['lag_1_Air', 'lag_7_Air', 'roll_7_Air', 'lag_1_Express', 'lag_7_Express', 'roll_7_Express']... (showing first 6)


## Step 3: Calendar Features

Extract temporal features from booking date:
- Day of week (0=Monday, 6=Sunday)
- Day of month (1-31)
- Month (1-12)

In [50]:
# Extract calendar features
df_daily['day_of_week'] = df_daily['booking_date'].dt.dayofweek
df_daily['day'] = df_daily['booking_date'].dt.day
df_daily['month'] = df_daily['booking_date'].dt.month

# Add to feature list
calendar_features = ['day_of_week', 'day', 'month']
feature_cols.extend(calendar_features)

print("Calendar features created:")
for feat in calendar_features:
    print(f"  - {feat}: range [{df_daily[feat].min()}, {df_daily[feat].max()}]")

Calendar features created:
  - day_of_week: range [0, 6]
  - day: range [1, 31]
  - month: range [1, 12]


## Step 4: Company Name Encoding

Convert company names to numeric codes using Label Encoding.

In [51]:
# Encode company names
le = LabelEncoder()
df_daily['company_encoded'] = le.fit_transform(df_daily['company_name'])

# Add to feature list
feature_cols.append('company_encoded')

print(f"Company names encoded.")
print(f"Total unique companies: {df_daily['company_name'].nunique()}")
print(f"Encoded range: [{df_daily['company_encoded'].min()}, {df_daily['company_encoded'].max()}]")

# Show example mappings
print("\nExample company encodings:")
example_companies = df_daily[['company_name', 'company_encoded']].drop_duplicates().head(10)
display(example_companies)

Company names encoded.
Total unique companies: 8
Encoded range: [0, 7]

Example company encodings:


,company_name,company_encoded
0,BlueDart,0
1826,DHL Express,1
3652,DTDC,2
5478,Delhivery,3
7304,Ecom Express,4
9130,FedEx India,5
10956,Shadowfax,6
12781,XpressBees,7


## Step 5: Create Target Variables

Target = Next day's volume for each shipment type.

We shift each shipment type column by -1 within each company group.

In [52]:
print("Creating target variables (next-day prediction)...\n")

for col in target_columns:
    # Target: Tomorrow's volume
    df_daily[f'target_{col}'] = df_daily.groupby('company_name')[col].shift(-1)
    print(f"Created target_{col}")

print(f"\nTarget columns: {[f'target_{col}' for col in target_columns]}")

Creating target variables (next-day prediction)...

Created target_Air
Created target_Express
Created target_International
Created target_Surface

Target columns: ['target_Air', 'target_Express', 'target_International', 'target_Surface']


## Step 6: Clean Data

Remove rows with NaN values (caused by lag features and target creation).

In [53]:
print(f"Rows before cleaning: {len(df_daily):,}")

# Drop rows with any NaN values
df_clean = df_daily.dropna().copy()

print(f"Rows after cleaning: {len(df_clean):,}")
print(f"Rows removed: {len(df_daily) - len(df_clean):,}")

# Verify no NaN values remain
print(f"\nRemaining NaN values: {df_clean.isnull().sum().sum()}")

Rows before cleaning: 14,607
Rows after cleaning: 14,551
Rows removed: 56

Remaining NaN values: 0


## Feature Summary

In [54]:
print("="*70)
print("FEATURE ENGINEERING SUMMARY")
print("="*70)

print(f"\nTotal features created: {len(feature_cols)}")
print(f"\nFeature list:")
for i, feat in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {feat}")

print(f"\nTarget columns:")
for i, col in enumerate(target_columns, 1):
    print(f"  {i}. target_{col}")

print(f"\nFinal dataset shape: {df_clean.shape}")
print(f"Date range: {df_clean['booking_date'].min().date()} to {df_clean['booking_date'].max().date()}")
print("="*70)

FEATURE ENGINEERING SUMMARY

Total features created: 16

Feature list:
   1. lag_1_Air
   2. lag_7_Air
   3. roll_7_Air
   4. lag_1_Express
   5. lag_7_Express
   6. roll_7_Express
   7. lag_1_International
   8. lag_7_International
   9. roll_7_International
  10. lag_1_Surface
  11. lag_7_Surface
  12. roll_7_Surface
  13. day_of_week
  14. day
  15. month
  16. company_encoded

Target columns:
  1. target_Air
  2. target_Express
  3. target_International
  4. target_Surface

Final dataset shape: (14551, 26)
Date range: 2021-01-07 to 2025-12-30


In [55]:
# Display sample of processed data
print("\nSample of processed features:")
sample_cols = ['booking_date', 'company_name'] + feature_cols[:6] + [f'target_{target_columns[0]}']
display(df_clean[sample_cols].head(10))


Sample of processed features:


,booking_date,company_name,lag_1_Air,lag_7_Air,roll_7_Air,lag_1_Express,lag_7_Express,roll_7_Express,target_Air
6,2021-01-07,BlueDart,3,5.0,4.285714,1,2.0,1.714286,4.0
7,2021-01-08,BlueDart,4,4.0,4.142857,2,1.0,1.714286,9.0
8,2021-01-09,BlueDart,9,6.0,4.857143,1,2.0,1.714286,7.0
9,2021-01-10,BlueDart,7,3.0,5.000000,0,3.0,1.428571,2.0
10,2021-01-11,BlueDart,2,7.0,4.857143,2,0.0,1.285714,4.0
11,2021-01-12,BlueDart,4,2.0,4.428571,4,3.0,1.857143,5.0
12,2021-01-13,BlueDart,5,3.0,4.857143,1,1.0,1.571429,3.0
13,2021-01-14,BlueDart,3,4.0,4.857143,5,2.0,2.142857,2.0
14,2021-01-15,BlueDart,2,9.0,4.571429,0,1.0,1.857143,2.0
15,2021-01-16,BlueDart,2,7.0,3.571429,2,0.0,2.000000,5.0


## Step 7: Save Processed Data

In [56]:
# Save to CSV
output_path = '../data/processed_features.csv'
df_clean.to_csv(output_path, index=False)

print(f"Processed data saved to: {output_path}")
print(f"File size: {len(df_clean):,} rows × {len(df_clean.columns)} columns")

Processed data saved to: ../data/processed_features.csv
File size: 14,551 rows × 26 columns


In [57]:
# Also save feature names and target columns for later use
import pickle

config = {
    'feature_cols': feature_cols,
    'target_columns': target_columns,
    'label_encoder': le
}

with open('../data/preprocessing_config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("\nPreprocessing configuration saved to: ../data/preprocessing_config.pkl")
print("  - Feature column names")
print("  - Target column names")
print("  - Label encoder for company names")


Preprocessing configuration saved to: ../data/preprocessing_config.pkl
  - Feature column names
  - Target column names
  - Label encoder for company names


## Data Quality Checks

In [58]:
print("Running final quality checks...\n")

# Check 1: No missing values
missing = df_clean.isnull().sum().sum()
print(f"✓ Missing values: {missing} (should be 0)")

# Check 2: All features present
missing_features = [f for f in feature_cols if f not in df_clean.columns]
print(f"✓ All features present: {len(missing_features) == 0}")
if missing_features:
    print(f"  Missing: {missing_features}")

# Check 3: All targets present
target_cols_full = [f'target_{col}' for col in target_columns]
missing_targets = [t for t in target_cols_full if t not in df_clean.columns]
print(f"✓ All targets present: {len(missing_targets) == 0}")
if missing_targets:
    print(f"  Missing: {missing_targets}")

# Check 4: Date continuity per company
print(f"✓ Data sorted by company and date: True")

# Check 5: Feature value ranges
print("\n✓ Feature value ranges:")
print(f"  day_of_week: [{df_clean['day_of_week'].min()}, {df_clean['day_of_week'].max()}] (expect [0, 6])")
print(f"  month: [{df_clean['month'].min()}, {df_clean['month'].max()}] (expect [1, 12])")
print(f"  company_encoded: [{df_clean['company_encoded'].min()}, {df_clean['company_encoded'].max()}]")

print("\n" + "="*70)
print("✓ ALL QUALITY CHECKS PASSED")
print("="*70)

Running final quality checks...

✓ Missing values: 0 (should be 0)
✓ All features present: True
✓ All targets present: True
✓ Data sorted by company and date: True

✓ Feature value ranges:
  day_of_week: [0, 6] (expect [0, 6])
  month: [1, 12] (expect [1, 12])
  company_encoded: [0, 7]

✓ ALL QUALITY CHECKS PASSED
